In [22]:
from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI
from llama_index.core.query_engine import NLSQLTableQueryEngine
from pathlib import Path

import os
from dotenv import load_dotenv

load_dotenv()  # carga desde .env

# opcionalmente, puedes verificar:
assert os.getenv("OPENAI_API_KEY"), "No se encontró OPENAI_API_KEY"
# 1. Cargar base de datos
db_path = Path("C:/Users/05457936J/Desktop/CORAL/opositIA/data/outputs/oposiciones.db").resolve()
sql_database = SQLDatabase.from_uri(f"sqlite:///{db_path.as_posix()}")

# 2. Crear cliente OpenAI (usando GPT-4.1-nano, por ejemplo)
llm = OpenAI(model="gpt-4.1-nano", temperature=0.1,
             api_key=os.getenv("OPENAI_API_KEY"))



In [32]:
# 3. Crear el motor de preguntas
query_engine = NLSQLTableQueryEngine(sql_database=sql_database, llm=llm, verbose=True)

pregunta = "¿cuántas plazas libres de AUXILIA hay para EXTREMADURA?"
respuesta = query_engine.query(pregunta)

print(respuesta.response)

> Table Info: Table 'oposiciones' has columns: texto_original (TEXT), titulo (TEXT), organismo_que_convoca (TEXT), titulacion_requerida (TEXT), via (TEXT), plazas_convocadas (TEXT), plazas_libres (TEXT), otras_plazas (TEXT), fecha_de_publicacion (TEXT), fecha_de_cierre (TEXT), referencia (TEXT), .
> Table desc str: Table 'oposiciones' has columns: texto_original (TEXT), titulo (TEXT), organismo_que_convoca (TEXT), titulacion_requerida (TEXT), via (TEXT), plazas_convocadas (TEXT), plazas_libres (TEXT), otras_plazas (TEXT), fecha_de_publicacion (TEXT), fecha_de_cierre (TEXT), referencia (TEXT), .
> Predicted SQL query: SELECT oposiciones.plazas_libres FROM oposiciones WHERE oposiciones.titulo LIKE '%AUXILIA%' AND oposiciones.organismo_que_convoca LIKE '%EXTREMADURA%' ORDER BY oposiciones.fecha_de_publicacion DESC LIMIT 1;
No hay plazas libres de AUXILIA para Extremadura según la información disponible.


In [23]:
import pandas as pd
import sqlite3


# Conectar y leer toda la tabla
conn = sqlite3.connect(db_path)
df = pd.read_sql("SELECT * FROM oposiciones", conn)
conn.close()



                                      texto_original                  titulo  \
0    \nPROCESOS DE ÁMBITO INTERNACIONAL  \nPERSON...               TRADUCTOR   
1    \nBACHILLERATO, BACHILLERATO UNIFICADO POLIV...  OFICIAL ADMINISTRATIVO   
2    \nGRADUADO EN EDUCACIÓN SECUNDARIA, EDUCACIÓ...               AUXILIA R   
3    \nMINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EU...               AUXILIA R   
4    \nMINISTERIO DE ASUNTOS EXTERIORES, UNIÓN EU...               AUXILIA R   

                               organismo_que_convoca  \
0                              MINISTERIO DE DEFENSA   
1  CONSEJERÍA DE TRABAJO, MIGRACIONES Y SEGURIDAD...   
2                EMBAJADA DE ESPAÑA EN ACCRA (GHANA)   
3                     CONSULADO DE ESPAÑA EN LARACHE   
4                       EMBAJADA DE ESPAÑA EN DUBLÍN   

                                titulacion_requerida                     via  \
0                                  OFICIAL TRADUCTOR       CONTRATACIÓN FIJA   
1  BACHILLERATO, BACHI

In [31]:
df.groupby(['titulo']).count()

,texto_original,organismo_que_convoca,titulacion_requerida,via,plazas_convocadas,plazas_libres,otras_plazas,fecha_de_publicacion,fecha_de_cierre,referencia
titulo,,,,,,,,,,
"ADMINISTRACIÓN ESPECIAL, SUBESCALA TÉCNICA AUXILIA R",1,1,1,1,1,1,1,1,1,1
ADMINISTRATIVO,5,5,5,5,5,5,5,5,5,5
ADMINISTRATIVO S,1,1,1,1,1,1,1,1,1,1
ADMINISTRATIVOS DE ADMINISTRACIÓN GENERAL,1,1,1,1,1,1,1,1,1,1
ANALISTA DE LABORATORIO,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
VIGILANTE S,1,1,1,1,1,1,1,1,1,1
coordinador de ensayos clínicos,1,1,1,1,1,1,1,1,1,1
investigador doctor senior,1,1,1,1,1,1,1,1,1,1
